In [ ]:
# from google.colab import drive # 挂载谷歌云盘
# drive.mount('/content/drive')
# !nvidia-smi # 显示显卡信息
# ''' 符号%代表一直生效，！代表执行完立马结束，不会生效，所以进入目录用% '''
# %cd /content/drive/MyDrive/timeSerise
# ''' 支持的 常用命令1.ls  2.wget  3.gdoint(int(int(int(w))))n  4.mkdir  5.pwd '''
# !ls
# !pip install patool
# !pip install sktime
# !pip install reformer_pytorch

In [1]:
import random
import numpy as np
import torch
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast
# from exp.exp_imputation import Exp_Imputation
# from exp.exp_short_term_forecasting import Exp_Short_Term_Forecast
# from exp.exp_anomaly_detection import Exp_Anomaly_Detection
# from exp.exp_classification import Exp_Classification
# from data_provider.data_creat import *
# import akshare as ak
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Args:
    '''基本配置'''
    # 选项：[long_term_forecast, short_term_forecast, imputation, classification, anomaly_detection]')
    task_name = 'long_term_forecast'
    is_training = 1 # 设置为1则进行训练，设置为0则进行测试
    model_id = 'stock_cy'
    des = '20231210' # 实验描述 20231210
    # 模型名称，选项：[Autoformer, Transformer, TimesNet]
    model = 'TimesNet'

    '''数据加载'''
    # 数据集类型,选项：[ETTh1,ETTh2,ETTm1,ETTm2,custom,m4,PSM,MSL,SMAP,SMD,SWAT,UEA]
    data = 'custom'
    test_path = 'raw_stock_cy_301327.csv'
    root_path = './dataset/Stock/'
    # 根据 is_training 的值设置 data_path
    if is_training == 1:
        data_path = 'stock_cy_all_raw.csv'
    else:
        data_path = test_path
    # 预测任务 M:多变量预测多变量, S:单变量预测单变量, MS:多变量预测单变量
    features = 'M'
    # 目标列名，S或MS任务中的目标特征
    target = 'Close'
    # 时间采集粒度，选项：[s:秒, t:分钟, h:小时, d:天, b:工作日, w:周, m:月]
    freq = 'd'
    # 模型检查点的位置
    checkpoints = './checkpoints/'

    '''预测任务'''
    # 输入序列长度,这是用于模型训练的输入序列的长度
    seq_len = 60
    # 开始标记长度,这是模型输出目标中有标签数据的长度，类似于滑动窗口的长度
    label_len = 20
    # 预测序列长度
    pred_len = 20
    # 季节模式（针对M4数据集）
    seasonal_patterns = 'Monthly'
    inverse = False    # 反转输出数据

    '''插补任务'''
    # 插补任务中数据丢失率
    mask_rate = 0.25

    '''异常检测任务'''
    # 异常检测中异常点占比
    anomaly_ratio = 0.25

    '''模型定义'''
    # TimesBlock 中傅里叶变换,频率排名前k个周期
    top_k = 5
    # Inception 中卷积核个数
    num_kernels = 6
    # encoder 输入特征数
    enc_in = 31
    # decoder 输入特征数
    dec_in = 31
    # 输出通道数
    c_out = 31
    # 线性层隐含神经元个数
    d_model = 32
    # FFN 层隐含神经元个数
    d_ff = 32
    # 多头注意力机制
    n_heads = 8
    # encoder 层数
    e_layers = 2
    # decoder 层数
    d_layers = 1
    # 滑动窗口长度
    moving_avg = 20
    # 对 Q 进行采样，对 Q 采样的因子数
    factor = 3
    # 是否下采样操作 pooling
    distil = True
    # dropout 率
    dropout = 0.1
    # 时间特征嵌入方式,选项：[timeF, fixed, learned]
    embed = 'timeF'
    # 激活函数类型
    activation = 'gelu'
    # 是否输出 attention
    output_attention = False

    '''优化'''
    # 测试集的比例
    if is_training == 1:
        test_ratio = [0.1,0.1] # test测试集、valid验证集的占比
    else:
        test_ratio = [0.6,0.1]

    # 并行核心数
    num_workers = 10
    # 实验轮数
    itr = 1
    # 训练迭代次数
    train_epochs = 500
    # batch size 大小
    batch_size = 256
    # early stopping 机制容忍次数
    patience = 5
    # 学习率
    learning_rate = 0.0001
    # 损失函数
    loss = 'MSE'
    # 学习率下降策略
    lradj = 'type1'
    # 使用混合精度训练
    use_amp = False

    '''GPU'''
    # 使用 gpu
    use_gpu = True
    gpu = 0
    # 使用多个 gpus
    use_multi_gpu = False
    # 多 gpu 的设备 id
    devices = '0,1,2,3'

    '''去平稳化投影仪参数'''
    # 投影仪的隐藏层维度（列表）
    p_hidden_dims = [128, 128]
    # 投影仪中的隐藏层数
    p_hidden_layers = 2


# 创建参数对象
args = Args()

# 设置随机种子以确保结果可重现
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)


# 获取列数
num_columns = 32
# args.des = NUM
args.enc_in = num_columns - 1
args.dec_in = num_columns - 1
args.c_out = num_columns - 1

# 检查并设置 GPU
args.use_gpu = torch.cuda.is_available() and args.use_gpu
if args.use_gpu:
    print("使用 GPU.")
    total_cuda_devices = torch.cuda.device_count()  # 获取系统中可用的 GPU 总数
    print(f"系统中总共有 {total_cuda_devices} 个 CUDA 设备可用。")
    if args.use_multi_gpu:
        args.devices = args.devices.replace(' ', '')
        device_ids = args.devices.split(',')
        args.device_ids = [int(id_) for id_ in device_ids]
        args.gpu = args.device_ids[0]

        # 打印多 GPU 使用情况
        print(f"使用多个GPU: {args.device_ids}")
        device = torch.device(f"cuda:{args.gpu}" if args.use_gpu else "cpu")
        print(f"Primary GPU (cuda:{args.gpu}) is in use.")
    else:
        args.gpu = 0
        device = torch.device("cuda" if args.use_gpu else "cpu")
        print("使用单个 GPU.")
else:
    device = torch.device("cpu")
    print("使用 CPU.")


# 选择合适的实验类
if args.task_name == 'long_term_forecast':
    Exp = Exp_Long_Term_Forecast
# elif args.task_name == 'short_term_forecast':
#     Exp = Exp_Short_Term_Forecast
# elif args.task_name == 'imputation':
#     Exp = Exp_Imputation
# elif args.task_name == 'anomaly_detection':
#     Exp = Exp_Anomaly_Detection
# elif args.task_name == 'classification':
#     Exp = Exp_Classification
else:
    Exp = Exp_Long_Term_Forecast  # 默认情况
    
    
ii = 0
setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.task_name, # 任务名称
            args.model_id, # 模型id
            args.model, # 模型名称
            args.data, # 数据集名称
            args.features, # 预测任务
            args.seq_len, # 输入序列长度
            args.label_len, # 开始标记长度
            args.pred_len, # 预测序列长度
            args.d_model, # encoder 输入特征数
            args.n_heads, # 多头注意力机制
            args.e_layers, # encoder 层数
            args.d_layers, # decoder 层数
            args.d_ff, # FFN 层隐含神经元个数
            args.factor, # 对 Q 采样的因子数
            args.embed, # 时间特征嵌入方式
            args.distil, # 是否下采样操作 pooling
            args.des,  # 实验描述
            ii) # 实验轮数

使用 GPU.
系统中总共有 1 个 CUDA 设备可用。
使用单个 GPU.


In [3]:
import os
import numpy as np
import pandas as pd
import glob
import re
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from utils.timefeatures import time_features
from data_provider.m4 import M4Dataset, M4Meta
from data_provider.uea import subsample, interpolate_missing, Normalizer
from sktime.datasets import load_from_tsfile_to_dataframe
import warnings

warnings.filterwarnings('ignore')

# A

In [ ]:
class Dataset_Custom(Dataset):
    '''
    root_path: 数据集文件所在的根目录路径。
    flag: 数据集的类型（'train'、'test' 或 'val'），默认为 'train'。
    size: 一个包含三个整数的元组，分别表示序列长度、标签长度和预测长度。
    features: 特征类型，'S' 表示单变量(single)，默认为 'S'。
    data_path: 数据文件的名称，默认为 'ETTh1.csv'。
    target: 目标变量的名称。
    scale: 布尔值，指示是否对数据进行标准化处理。
    timeenc: 时间编码类型(0 或 1),用于处理时间特征。
    freq: 时间频率。
    seasonal_patterns: 季节性模式（如果有）。
    '''
    def __init__(self, root_path, flag='train', size=None,
                 features='S', data_path='ETTh1.csv',
                 target='OT', scale=True, timeenc=0, freq='h', test_ratio=0.2 , seasonal_patterns=None):
        # size [seq_len, label_len, pred_len]
        # info
        if size == None:
            self.seq_len = 24 * 4 * 4
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]
        # init
        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]

        self.features = features
        self.target = target
        self.scale = scale
        self.timeenc = timeenc
        self.freq = freq

        self.root_path = root_path
        self.data_path = data_path
        self.test_ratio = test_ratio
        self.__read_data__()

    def __read_data__(self):
        '''
        读取数据长度,区分train、test、vali三部分数据的边界
        '''
        # 数据标准化实例
        self.scaler = StandardScaler()
        # 读取数据
        df_raw = pd.read_csv(os.path.join(self.root_path,
                                          self.data_path))

        '''
        df_raw.columns: ['date', ...(other features), target feature]
        '''
        cols = list(df_raw.columns)
        cols.remove(self.target) # 移除目标特征
        cols.remove('date') # 移除日期特征
        df_raw = df_raw[['date'] + cols + [self.target]] # 重新排序以后的数据
        
        # 数据拆分比例
        num_train = int(len(df_raw) * (1-self.test_ratio[0]-self.test_ratio[1]))
        num_test = int(len(df_raw) * self.test_ratio[0])
        num_vali = len(df_raw) - num_train - num_test
        
        # 计算train、test、vali数据起始点，也就是拆分完的纵向数据长度
        border1s = [0, num_train - self.seq_len, len(df_raw) - num_test - self.seq_len]  # [0，train长度-60，train+valid长度-60 ]
        border2s = [num_train, num_train + num_vali, len(df_raw)] # [train长度，train+valid长度，原始长度]
        border1 = border1s[self.set_type] # set_type {'train': 0, 'val': 1, 'test': 2} 减去输入序列长度
        border2 = border2s[self.set_type] # set_type {'train': 0, 'val': 1, 'test': 2}

        # 如果预测对象为多变量预测或多元预测单变量，那么取除日期外得所有列为df_data
        if self.features == 'M' or self.features == 'MS':
            cols_data = df_raw.columns[1:]
            df_data = df_raw[cols_data]
        # 若预测类型为S(单特征预测单特征)
        elif self.features == 'S':
            df_data = df_raw[[self.target]]

        # 原始全部数据归一化
        if self.scale:
            train_data = df_data[border1s[0]:border2s[0]] # 训练数据
            self.scaler.fit(train_data.values) # 训练数据归一化
            data = self.scaler.transform(df_data.values) # 全部数据归一化
        else:
            data = df_data.values  # 不归一化
        
        # 处理日期格式
        df_stamp = df_raw[['date']][border1:border2] # {'train': 0, 'val': 1, 'test': 2}
        df_stamp['date'] = pd.to_datetime(df_stamp.date)# 利用pandas将数据转换为日期格式
        if self.timeenc == 0: # 构建时间特征
            df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
            df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
            df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
            df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
            data_stamp = df_stamp.drop(['date'], 1).values
        elif self.timeenc == 1:
            data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=self.freq) # 时间特征构造函数，用来提取日期特征
            data_stamp = data_stamp.transpose(1, 0)            # 转置
        
        # 计算train、test、vali的 X和Y部分数据长度，是相等的长度 
        self.data_x = data[border1:border2] # {'train': 0, 'val': 1, 'test': 2}
        self.data_y = data[border1:border2] # {'train': 0, 'val': 1, 'test': 2}
        self.data_stamp = data_stamp
        
    def __getitem__(self, index):
        '''
        区分特征X在第几列  和 标签Y在第几列
        label_len 滑动窗口长度 20
        seq_len 输入序列长度 60
        
        预测的基本思想是将已知序列长度延长到 (seq_len+pred_len)，这是预测后的总长度。
        '''
        # 取得起始标签
        s_begin = index                                  # X 的开始位置。n
        s_end = s_begin + self.seq_len                   # X 的结束位置。开始标签 到 输入序列长度 的区间 n+60=n+60
        r_begin = s_end - self.label_len                 # Y 的开始位置。X结束位置 减去 滑动窗口长度 n+60-20=n+40  从第40个开始是第一个Y
        r_end = r_begin + self.label_len + self.pred_len # Y 的结束位置。Y开始位置 加上 滑动窗口长度 加上 预测长度n+40+20+1=n+61

        # 取训练数据
        seq_x = self.data_x[s_begin:s_end] # 0：60
        seq_y = self.data_y[r_begin:r_end] # 40：61
        # 取训练数据对应时间特征
        seq_x_mark = self.data_stamp[s_begin:s_end]
        # 取有标签区间+无标签区间(预测时间步长)对应时间特征
        seq_y_mark = self.data_stamp[r_begin:r_end]

        return seq_x, seq_y, seq_x_mark, seq_y_mark

    def __len__(self):
        return len(self.data_x) - self.seq_len - self.pred_len + 1  # 0：60  60：61  61：61

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)  # 反归一化


# B

In [14]:
import os
import torch
from models import Autoformer, Transformer, TimesNet, Nonstationary_Transformer, DLinear, FEDformer, \
    Informer, LightTS, Reformer, ETSformer, Pyraformer, PatchTST, MICN, Crossformer, FiLM, iTransformer, \
    Koopa


class Exp_Basic(object):
    def __init__(self, args):
        self.args = args
        self.model_dict = {
            'TimesNet': TimesNet,
            'Autoformer': Autoformer,
            'Transformer': Transformer,
            'Nonstationary_Transformer': Nonstationary_Transformer,
            'DLinear': DLinear,
            'FEDformer': FEDformer,
            'Informer': Informer,
            'LightTS': LightTS,
            'Reformer': Reformer,
            'ETSformer': ETSformer,
            'PatchTST': PatchTST,
            'Pyraformer': Pyraformer,
            'MICN': MICN,
            'Crossformer': Crossformer,
            'FiLM': FiLM,
            'iTransformer': iTransformer,
            'Koopa': Koopa,
        }
        self.device = self._acquire_device()
        self.model = self._build_model().to(self.device)

    def _build_model(self):
        raise NotImplementedError
        return None

    def _acquire_device(self):
        if self.args.use_gpu:
            os.environ["CUDA_VISIBLE_DEVICES"] = str(
                self.args.gpu) if not self.args.use_multi_gpu else self.args.devices
            device = torch.device('cuda:{}'.format(self.args.gpu))
            print('Use GPU: cuda:{}'.format(self.args.gpu))
        else:
            device = torch.device('cpu')
            print('Use CPU')
        return device

    def _get_data(self):
        pass

    def vali(self):
        pass

    def train(self):
        pass

    def test(self):
        pass
    
    def predict(self):
        pass

In [13]:
class Exp_Long_Term_Forecast(Exp_Basic):
    def __init__(self, args):
        super(Exp_Long_Term_Forecast, self).__init__(args)

    def _build_model(self):
        model = self.model_dict[self.args.model].Model(self.args).float()

        if self.args.use_multi_gpu and self.args.use_gpu:
            model = nn.DataParallel(model, device_ids=self.args.device_ids)
        return model

    def _get_data(self, flag): # 加载数据
        data_set, data_loader = data_provider(self.args, flag)
        return data_set, data_loader

    def _select_optimizer(self):
        model_optim = optim.Adam(self.model.parameters(), lr=self.args.learning_rate)
        return model_optim

    def _select_criterion(self):
        criterion = nn.MSELoss()
        return criterion

    def vali(self, vali_data, vali_loader, criterion):
        total_loss = []
        self.model.eval()
        with torch.no_grad():
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(vali_loader):
                batch_x = batch_x.float().to(self.device)
                batch_y = batch_y.float()

                batch_x_mark = batch_x_mark.float().to(self.device)
                batch_y_mark = batch_y_mark.float().to(self.device)

                # decoder input
                dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float()
                dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)
                # encoder - decoder
                if self.args.use_amp:
                    with torch.cuda.amp.autocast():
                        if self.args.output_attention:
                            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                        else:
                            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                else:
                    if self.args.output_attention:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                    else:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                f_dim = -1 if self.args.features == 'MS' else 0
                outputs = outputs[:, -self.args.pred_len:, f_dim:]
                batch_y = batch_y[:, -self.args.pred_len:, f_dim:].to(self.device)

                pred = outputs.detach().cpu()
                true = batch_y.detach().cpu()

                loss = criterion(pred, true)

                total_loss.append(loss)
        total_loss = np.average(total_loss)
        self.model.train()
        return total_loss

    def train(self, setting):
        # 取得训练、验证、测试数据及数据加载器
        writer = SummaryWriter(log_dir=os.path.join('tensorboard_logs', setting))
        train_data, train_loader = self._get_data(flag='train')
        vali_data, vali_loader = self._get_data(flag='val')
        test_data, test_loader = self._get_data(flag='test')

        path = os.path.join(self.args.checkpoints, setting)
        if not os.path.exists(path):
            os.makedirs(path)

        time_now = time.time()

        # 取训练步数
        train_steps = len(train_loader)
        # 设置早停参数
        early_stopping = EarlyStopping(patience=self.args.patience, verbose=True)

        # 选择优化器
        model_optim = self._select_optimizer()
        # 选择损失函数
        criterion = self._select_criterion()

        # 如果多GPU并行
        if self.args.use_amp:
            scaler = torch.cuda.amp.GradScaler()

        # 训练次数
        for epoch in range(self.args.train_epochs):
            iter_count = 0
            train_loss = []

            self.model.train() # 将模型设置为训练模式.
            epoch_time = time.time()
            train_pbar = tqdm(train_loader, position=0, leave=True) # 载入进度条
            
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(train_pbar):
                iter_count += 1
                # 梯度归零
                model_optim.zero_grad()
                
                # 取训练数据
                batch_x = batch_x.float().to(self.device)

                batch_y = batch_y.float().to(self.device)
                batch_x_mark = batch_x_mark.float().to(self.device)
                batch_y_mark = batch_y_mark.float().to(self.device)

                # decoder input
                dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float()
                dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)

                # encoder - decoder 并行计算
                if self.args.use_amp:
                    with torch.cuda.amp.autocast():
                        if self.args.output_attention:
                            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                        else:
                            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

                        f_dim = -1 if self.args.features == 'MS' else 0
                        outputs = outputs[:, -self.args.pred_len:, f_dim:]
                        batch_y = batch_y[:, -self.args.pred_len:, f_dim:].to(self.device)
                        loss = criterion(outputs, batch_y)
                        train_loss.append(loss.item())
                else:
                    if self.args.output_attention:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                    else:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

                    # 如果预测方式为MS，取最后1列否则取第1列
                    f_dim = -1 if self.args.features == 'MS' else 0
                    outputs = outputs[:, -self.args.pred_len:, f_dim:]
                    batch_y = batch_y[:, -self.args.pred_len:, f_dim:].to(self.device)
                    # 计算损失
                    loss = criterion(outputs, batch_y)
                    # 将损失放入train_loss列表中
                    train_loss.append(loss.item())

                # 记录训练过程
                if (i + 1) % 100 == 0:
                    print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
                    speed = (time.time() - time_now) / iter_count
                    left_time = speed * ((self.args.train_epochs - epoch) * train_steps - i)
                    print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
                    iter_count = 0
                    time_now = time.time()

                if self.args.use_amp:
                    scaler.scale(loss).backward()
                    scaler.step(model_optim)
                    scaler.update()
                else:
                    # 反向传播
                    loss.backward()
                    # 更新梯度
                    model_optim.step()
            # 在 tqdm 进度条上显示当前 epoch number和loss .
            train_pbar.set_description(f'Epoch [{epoch+1}/{self.args.train_epochs}]') # 设置进度条的前缀
            train_pbar.set_postfix({'loss' : loss.detach().item()}) # 设置进度条的后缀

            print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
            train_loss = np.average(train_loss) # 计算平均损失
            writer.add_scalar('Loss/train_epoch', train_loss, epoch) # 将训练损失写入tensorboard
            print("正在计算验证集Loss...")
            vali_loss = self.vali(vali_data, vali_loader, criterion) # 计算验证损失
            writer.add_scalar('Loss/val', vali_loss, epoch) # 将验证损失写入tensorboard
            print("正在计算测试集Loss...")
            test_loss = self.vali(test_data, test_loader, criterion) # 计算测试损失

            print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f} Vali Loss: {3:.7f} Test Loss: {4:.7f}".format(
                epoch + 1, train_steps, train_loss, vali_loss, test_loss))
            early_stopping(vali_loss, self.model, path)
            if early_stopping.early_stop:
                print("Early stopping")
                break

            # 更新学习率
            adjust_learning_rate(model_optim, epoch + 1, self.args)

        # 保存模型
        best_model_path = path + '/' + 'checkpoint.pth'
        self.model.load_state_dict(torch.load(best_model_path))
        writer.close()

        return self.model

In [10]:
exp = Exp(args) # 创建实验对象
print('>>>>>>>开始训练 : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
exp.train(setting) # 训练模型

Use CPU
>>>>>>>开始训练 : long_term_forecast_stock_cy_TimesNet_custom_ftM_sl60_ll20_pl20_dm32_nh8_el2_dl1_df32_fc3_ebtimeF_dtTrue_20231210_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1407
val 168
test_batch_size:  256
test 166


  0%|          | 0/5 [00:30<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# 进行训练和测试
if args.is_training:
    # 多次训练模型
    for ii in range(args.itr):
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.task_name, # 任务名称
            args.model_id, # 模型id
            args.model, # 模型名称
            args.data, # 数据集名称
            args.features, # 预测任务
            args.seq_len, # 输入序列长度
            args.label_len, # 开始标记长度
            args.pred_len, # 预测序列长度
            args.d_model, # encoder 输入特征数
            args.n_heads, # 多头注意力机制
            args.e_layers, # encoder 层数
            args.d_layers, # decoder 层数
            args.d_ff, # FFN 层隐含神经元个数
            args.factor, # 对 Q 采样的因子数
            args.embed, # 时间特征嵌入方式
            args.distil, # 是否下采样操作 pooling
            args.des,  # 实验描述
            ii) # 实验轮数

        exp = Exp(args) # 创建实验对象
        print('>>>>>>>开始训练 : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting) # 训练模型

        print('>>>>>>>测试 : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting) # 测试模型
        if args.use_gpu:
            torch.cuda.empty_cache()
else:
    # 模型测试
    ii = 0
    setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.task_name,
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des,
        ii)
    exp = Exp(args)  # 创建实验对象
    print('>>>>>>>测试 : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting, test=1) # 测试模型
    if args.use_gpu:
        torch.cuda.empty_cache()

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./tensorboard_logs/